   # Procesing Tweets and Retweets Dataset and loading into MongoDB database system

### Following constrains are used to reduce the querrying time


1) Irrelevant  data fields are dropped.
 

2) Order of appearance of attributes is changed.

3) a compound index is created on "id_str user's sub-attribute " id_str" 

### Importing reuired libraried

In [7]:
import os
import json
import tweepy 
import pandas as pd 
import pymongo
import time
import datetime
from datetime import datetime, date

###                                       Setting up mongoDB client connection 

In [8]:
client=pymongo.MongoClient("mongodb://localhost:27017")

### Creating an empty database

In [9]:
db=client["corona_3"]

In [10]:
#this function is used to parse date in YYYY-MM-DD format in created_at attribute of both, the post and the user.


def parse_date(date_string):
    # parse the created_at attributes to datetime formart
    
    date_format = "%a %b %d %H:%M:%S %z %Y"
    date_object = time.strptime(date_string, date_format)
    
    year = date_object.tm_year
    month = date_object.tm_mon
    day = date_object.tm_mday
    
    new_date_object = datetime(year=year, month=month, day=day)
    formatted_date = new_date_object.strftime("%Y-%m-%d")
    
    return(formatted_date)

In [11]:
# This is used to store tweets in database
with open('corona-out-3.json', 'r') as f:
    print("Inserting tweets in MongoDB")
    retweeted_status_keys=["created_at","id_str","text","in_reply_to_status_id_str","in_reply_to_user_id_str","in_reply_to_screen_name","user","quoted_status_id_str","quote_count","reply_count","retweet_count","favorite_count","entities","lang"]
    # Process tweets
    for line in f:
        if(line=='\n'):
            continue            
        tweet = json.loads(line)
        dictz = {"user":None,'text':None,"entities":None,'created_at': None, 'id_str':None,"in_reply_to_screen_name":None ,"in_reply_to_user_id_str": None,"in_reply_to_status_id_str": None,"quoted_status_id_str": None, "quote_count" : None,"reply_count" : None,"retweet_count": None,"favorite_count": None, "lang": None}
        
        
        # Check if tweet has not been retweeted
        if "retweeted_status" not in tweet:
            # Extract information from tweet
            for line2 in tweet:
                if line2 in dictz:
                    dictz[line2]=tweet[line2]
            if "created_at" in tweet:
                created_at=tweet["user"]
                dictz["created_at"]=parse_date(tweet["created_at"])
            if "user" in tweet:
                user=tweet["user"]
               
                 
                dictz["user"]={"id_str":user["id_str"],"name":user["name"],"screen_name":user["screen_name"],"followers_count":user["followers_count"],"favourites_count":user["favourites_count"],"statuses_count":user["statuses_count"]}     
            
                    
            # Insert tweet into MongoDB collection
            db.tweets.insert_one(dictz)
    print("tweets inserted")
    print("complete")

Inserting tweets in MongoDB
tweets inserted
complete


### Creating an compound index on id_str and user's sub-attribute id_str of tweets collection


In [5]:
db.tweets.create_index([("user.id_str", 1), ("id_str", 1)])

'user.id_str_1_id_str_1'

In [20]:
# This is used to store retweets in database

with open('corona-out-2.json', 'r') as f:
    print("Inserting Retweets in MongoDB")
    retweeted_status_keys=["id_str","text","in_reply_to_status_id_str","in_reply_to_user_id_str","in_reply_to_screen_name","quoted_status_id_str","quote_count","reply_count","retweet_count","favorite_count","entities","lang"]
    # Process tweets
    for line in f:
        if(line=='\n'):
            continue            
        retweet = json.loads(line)
        dictz = {"user":None,'text':None,"entities":None,'created_at': None, 'id_str':None,"in_reply_to_screen_name":None ,"in_reply_to_user_id_str": None,"in_reply_to_status_id_str": None,"quoted_status_id_str": None, "quote_count" : None,"reply_count" : None,"retweet_count": None,"favorite_count": None, "lang": None}
        
        # Check if tweet has not been retweeted
        if "retweeted_status" in retweet:
            rts= tweet["retweeted_status"]



            retweeted_status={}                 
            for key in retweeted_status_keys:
                if key in rts: 

                    retweeted_status[key] = rts[key]                   
                dictz['retweeted_status'] = retweeted_status
            # Extract information from tweet
            for line2 in tweet:
                if line2 in dictz:
                    dictz[line2]=tweet[line2]
            if "created_at" in tweet:
                created_at=tweet["user"]
                dictz["created_at"]=parse_date(tweet["created_at"])
            if "user" in retweet:
                user=tweet["user"]           
                dictz["user"]={"id_str":user["id_str"],"created_at":parse_date(user["created_at"])}                  
                    
            
            # Insert retweet into MongoDB collection
            db.retweets.insert_one(dictz)
    
    
    
    print("Retweets inserted")
    print("complete")

Inserting Retweets in MongoDB
Retweets inserted
complete


### Creating an compound index on id_str and user's sub-attribute id_str of retweets collection

In [6]:
db.retweets.create_index([("user.id_str", 1), ("id_str", 1)])

'user.id_str_1_id_str_1'